# Conversational Agent

## Basics

### Agents

- They are a combination of LLMs and code.
- The LLM reasons about what steps to take and call for actions.

### Agent loop

- Choose a `tool` to use
- Observe the `output` of the `tool`
- Repeat the loop until a `stopping condition` is met

### Stoping conditions

- LLM determined
- Hardcoded rules (max number of iterations, max time, etc)

### In this notebook we are going to

- Build some tools
- Write your own agent loop using LCEL
- Utilize LangChain's `agent_executor` class which:
  - Implements the agent loop
  - Adds error handling, early stopping, tracing, etc.


In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

import json


_ = load_dotenv(find_dotenv())  # read local .env file
openai.api_key = os.environ["OPENAI_API_KEY"]

In [2]:
from tools.weather import get_current_temperature
from tools.wikipedia import search_wikipedia

from langchain.chat_models import ChatOpenAI
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function

In [3]:
tools = [get_current_temperature, search_wikipedia]

In [4]:
functions = [format_tool_to_openai_function(function) for function in tools]

model = ChatOpenAI(temperature=0).bind(functions=functions)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are helpful but sassy assistent"),
        ("user", "{input}"),
    ]
)

chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [5]:
result = chain.invoke({"input": "What is the weather in Berlin?"})

In [6]:
result.tool

'get_current_temperature'

In [7]:
result.tool_input

{'latitude': 52.52, 'longitude': 13.405}

## Agent Memory 101

We want a place where we can pass in a list of messages.

We are going to convert this tool choice into a tool observation and into a list of messages and pass it back in.

For the list of messages we are going to use `MessagesPlaceholder`, and the placeholder will be called `agent_scratchpad`.

So the logic will be of:

- A system message
- The user message
- Action-Observation pairs


In [8]:
from langchain.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are helpful but sassy assistent"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [9]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

At this point we pass an empty list to the `agent_scratchpad` placeholder because we haven't messaged the model yet.


In [10]:
model_result_with_empty_agent_scratchpad = chain.invoke(
    {
        "input": "What is the weather in Berlin?",
        "agent_scratchpad": [],
    }
)

In [11]:
model_result_with_empty_agent_scratchpad.tool

'get_current_temperature'

In [12]:
model_observation = get_current_temperature(
    model_result_with_empty_agent_scratchpad.tool_input
)

In [13]:
model_observation

'The current temperature is 7.6°C'

In [14]:
type(model_result_with_empty_agent_scratchpad)

langchain_core.agents.AgentActionMessageLog

The next step is to convert the tool's observation back into a list of messages that we can pass into the `agent_scratchpad`.

How do we do that?

Using the function `format_to_openai_functions`.

---

**Behind the scenes of `format_to_openai_functions`**

We take the model's observation `message_log` value.

The `message_log` is a list of messages that describes how we arrived at this current age and action.

It stores values such as the called `function`, the `arguments` to that function with the OpenAI's `response`.

---

About the `observation` we are going to use the `function message` type from Lesson 1.


In [15]:
from langchain.agents.format_scratchpad import format_to_openai_functions

In [16]:
model_result_with_empty_agent_scratchpad.message_log

[AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_temperature', 'arguments': '{\n  "latitude": 52.5200,\n  "longitude": 13.4050\n}'}})]

`format_to_openai_functions` will be called on a list of tuples, corresponding to an Agent's `action` and an `observation`.

Why a **list of tuples**?

Because if we add more steps for the agent to take, we can just add more tuples to the list.


In [17]:
format_to_openai_functions(
    [(model_result_with_empty_agent_scratchpad, model_observation)]
)

[AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_temperature', 'arguments': '{\n  "latitude": 52.5200,\n  "longitude": 13.4050\n}'}}),
 FunctionMessage(content='The current temperature is 7.6°C', name='get_current_temperature')]

In [18]:
model_result_with_filled_agent_scratchpad = chain.invoke(
    {
        "input": "What is the weather in Berlin?",
        "agent_scratchpad": format_to_openai_functions(
            [(model_result_with_empty_agent_scratchpad, model_observation)]
        ),
    }
)

In [19]:
model_result_with_filled_agent_scratchpad

AgentFinish(return_values={'output': 'The current temperature in Berlin is 7.6°C.'}, log='The current temperature in Berlin is 7.6°C.')

In [25]:
from langchain.schema.runnable import RunnablePassthrough
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | chain

In [26]:
from langchain.schema.agent import AgentFinish
def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = chain.invoke({
            "input": user_input, 
            "agent_scratchpad": format_to_openai_functions(intermediate_steps)
        })
        if isinstance(result, AgentFinish):
            return result
        tool = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }[result.tool]
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result, observation))

In [27]:
run_agent("what is the weather is sf?")

AgentFinish(return_values={'output': 'The current temperature in San Francisco is 15.8°C.'}, log='The current temperature in San Francisco is 15.8°C.')

In [28]:
run_agent("what is langchain?")

AgentFinish(return_values={'output': 'I couldn\'t find specific information about "LangChain" in my search results. It\'s possible that LangChain is a relatively new or niche concept that is not widely documented. If you have any additional information or context about LangChain, I may be able to provide more assistance.'}, log='I couldn\'t find specific information about "LangChain" in my search results. It\'s possible that LangChain is a relatively new or niche concept that is not widely documented. If you have any additional information or context about LangChain, I may be able to provide more assistance.')

In [29]:
run_agent("hi!")

AgentFinish(return_values={'output': 'Hello! How can I assist you today?'}, log='Hello! How can I assist you today?')

In [30]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True)

In [31]:
agent_executor.invoke({"input": "what is langchain?"})



> Entering new AgentExecutor chain...

Invoking: `search_wikipedia` with `{'query': 'langchain'}`


Page: LangChain
Summary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.

Page: OpenAI
Summary: OpenAI is a U.S. artificial intelligence (AI) research organization founded in December 2015, researching artificial intelligence with the declared intention of developing "safe and beneficial" artificial general intelligence, which it defines as "highly autonomous systems that outperform humans at most economically valuable work".
As one of the leading organizations of the AI Spring, it has developed several large language models, advanced image generation models, and previously, also open-source models. Its release of ChatGPT has be

{'input': 'what is langchain?',
 'output': 'I couldn\'t find specific information about "LangChain" in my search results. However, there is a framework called "LangChain" that is designed to simplify the creation of applications using large language models (LLMs). It is used for tasks such as document analysis and summarization, chatbots, and code analysis.\n\nIf you were referring to something else, please provide more context or clarify your question.'}

In [32]:
agent_executor.invoke({"input": "my name is bob"})



> Entering new AgentExecutor chain...
Hello Bob! How can I assist you today?

> Finished chain.


{'input': 'my name is bob', 'output': 'Hello Bob! How can I assist you today?'}

In [33]:
agent_executor.invoke({"input": "what is my name"})



> Entering new AgentExecutor chain...
I'm sorry, but I don't have access to personal information.

> Finished chain.


{'input': 'what is my name',
 'output': "I'm sorry, but I don't have access to personal information."}

In [34]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [35]:
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | prompt | model | OpenAIFunctionsAgentOutputParser()

In [36]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")

In [37]:
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True, memory=memory)

In [38]:
agent_executor.invoke({"input": "my name is bob"})



> Entering new AgentExecutor chain...
Hello Bob! How can I assist you today?

> Finished chain.


{'input': 'my name is bob',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content='Hello Bob! How can I assist you today?')],
 'output': 'Hello Bob! How can I assist you today?'}

In [39]:
agent_executor.invoke({"input": "whats my name"})



> Entering new AgentExecutor chain...
Your name is Bob.

> Finished chain.


{'input': 'whats my name',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content='Hello Bob! How can I assist you today?'),
  HumanMessage(content='whats my name'),
  AIMessage(content='Your name is Bob.')],
 'output': 'Your name is Bob.'}

In [40]:
agent_executor.invoke({"input": "whats the weather in sf?"})



> Entering new AgentExecutor chain...

Invoking: `get_current_temperature` with `{'latitude': 37.7749, 'longitude': -122.4194}`


The current temperature is 15.8°CThe current temperature in San Francisco is 15.8°C.

> Finished chain.


{'input': 'whats the weather in sf?',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content='Hello Bob! How can I assist you today?'),
  HumanMessage(content='whats my name'),
  AIMessage(content='Your name is Bob.'),
  HumanMessage(content='whats the weather in sf?'),
  AIMessage(content='The current temperature in San Francisco is 15.8°C.')],
 'output': 'The current temperature in San Francisco is 15.8°C.'}